In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Any results you write to the current directory are saved as output.

/kaggle/input/breast-cancer-wisconsin-data/data.csv


In [94]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
display(df.head())
display(df.info())

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

None

** TABLE OF CONTENTS **

1. [Cleaning the dataset](#section1)

2. [Distribution of features across malignant and benign classes](#section2)

3. [Split into train and test sets](#section3) 

4. [Logistic Regression](#section4) 

5. [Decision Tree](#section5) 

6. [Random Forest Classifier](#section6) 

7. [XGBoost Classifier](#section7) 

8. [Conclusion](#section8)

**Dataset is available from **UCI ML** https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)


![](http://)### 1. Cleaning the dataset <a id='section1'> </a>

In [95]:
# columns to drop
df.columns[0], df.columns[-1]
df.drop(columns=['id', 'Unnamed: 32'], axis=1, inplace=True)
# class lables
df.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [96]:
# use labelencoder to convert the labels to '1' and '0'
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit_transform(df['diagnosis'])

# change the labels to '1' and '0'
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

In [97]:
px.histogram(df, x= 'diagnosis', color='diagnosis', hover_data=df.columns)

In [98]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [99]:
display(df.columns)

# feature to examine for variations in either clases
examine = df.columns[1:11]
display(examine)

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean'],
      dtype='object')

In [100]:
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly_white'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [101]:
pio.templates.default='plotly_white'

### 2. Distribution of features across **malignant** and **benign** classes <a id='section2'> </a>

In [109]:
dfM = df[df['diagnosis']==1]
dfB = df[df['diagnosis']==0]

for i in examine:
    fig = go.Figure()
    fig.add_trace(go.Histogram(
                x = dfM['radius_mean'],
                name='malignant',
                marker_color='red',
                xbins=dict(size=0.5)
                ))
    fig.add_trace(go.Histogram(
                x = dfB['radius_mean'],
                name='benign',
                marker_color='blue',
                xbins=dict(size=0.5)
                ))
    fig.update_layout(barmode='overlay', title_text='Distribution of feature: '+str(i) + ' for malignant and benign cases')
#reduce opacity
    fig.update_traces(opacity=0.75)
    fig.show()

### 3. Split into train and test sets <a id='section3'></a>

In [103]:
from sklearn.model_selection import train_test_split
# 70% train, 30% test
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

### 4. Logistic Regression <a id="section4"></a>

In [104]:
examine.tolist()

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean']

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
#logreg = LogisticRegression()
logreg = linear_model.LogisticRegressionCV(cv=10,verbose=1,n_jobs=-1,scoring='accuracy',solver='lbfgs')

features=examine.tolist()
to_predict= 'diagnosis'

logreg.fit(df_train[features], df_train['diagnosis'])
prediction = logreg.predict(df_test[features])
prediction

from sklearn import metrics
metrics.accuracy_score(prediction, df_test['diagnosis'])


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.0s finished


0.9473684210526315

In [153]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

def classification_models(model, df, df_train, df_test, cv=7, features=None):
    if features==None:
        model.fit(df_train[examine], df_train['diagnosis'])
        # prediction on the test
        prediction = model.predict(df_test[examine])
    else:
        model.fit(df_train[features], df_train['diagnosis'])
        # prediction on the test
        prediction = model.predict(df_test[features])
    
    # prediction on the test
    #prediction = model.predict(df_test[examine])
    
    # accuracy
    accuracy = accuracy_score(prediction, df_test['diagnosis'])
    print('Accuracy for model: {} is {}'.format(model, accuracy))
    
    # cross validation
    score= cross_val_score(estimator=model, X = df[examine], y=df['diagnosis'], cv=7, n_jobs=-1)
    for i in range(len(score)):
        print('Cross_validation score for for fold:{} is {}'.format(i,round(score[i],2)))

In [154]:
#from sklearn.exceptions import ConvergenceWarning
logreg = LogisticRegression()
classification_models(logreg, df, df_train, df_test, cv=7)

Accuracy for model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) is 0.935672514619883
Cross_validation score for for fold:0 is 0.91
Cross_validation score for for fold:1 is 0.85
Cross_validation score for for fold:2 is 0.89
Cross_validation score for for fold:3 is 0.94
Cross_validation score for for fold:4 is 0.93
Cross_validation score for for fold:5 is 0.94
Cross_validation score for for fold:6 is 0.91


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



### 5. Decision Tree <a id ='section5'></a>

In [155]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

classification_models(dtc, df, df_train, df_test, cv=7)
# for depth in range(3,7):
#     dtc = DecisionTreeClassifier(max_depth=depth)
#     classification_models(dtc, df, df_train, df_test, cv=7)

Accuracy for model: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') is 0.9122807017543859
Cross_validation score for for fold:0 is 0.93
Cross_validation score for for fold:1 is 0.84
Cross_validation score for for fold:2 is 0.89
Cross_validation score for for fold:3 is 0.9
Cross_validation score for for fold:4 is 0.94
Cross_validation score for for fold:5 is 0.93
Cross_validation score for for fold:6 is 0.94


### 6. Random Forest Classifier <a id='section6'></a>

In [156]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150, n_jobs=-1, min_samples_split=10)

classification_models(rf, df, df_train, df_test, cv=7)

Accuracy for model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False) is 0.9473684210526315
Cross_validation score for for fold:0 is 0.93
Cross_validation score for for fold:1 is 0.87
Cross_validation score for for fold:2 is 0.94
Cross_validation score for for fold:3 is 0.95
Cross_validation score for for fold:4 is 0.95
Cross_validation score for for fold:5 is 0.95
Cross_validation score for for fold:6 is 0.98


In [123]:
#rf.feature_importances_, features
imp_features =pd.Series(rf.feature_importances_, index= features).sort_values(ascending=False)
imp_features

concave points_mean       0.302025
concavity_mean            0.172393
area_mean                 0.146885
perimeter_mean            0.120138
radius_mean               0.103131
texture_mean              0.056170
compactness_mean          0.050221
smoothness_mean           0.019065
fractal_dimension_mean    0.015638
symmetry_mean             0.014334
dtype: float64

In [157]:
#top 5 features
display(imp_features.index[:5].tolist())
classification_models(rf, df, df_train, df_test, cv=7, features=imp_features.index[:5].tolist())

['concave points_mean',
 'concavity_mean',
 'area_mean',
 'perimeter_mean',
 'radius_mean']

Accuracy for model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False) is 0.9181286549707602
Cross_validation score for for fold:0 is 0.93
Cross_validation score for for fold:1 is 0.87
Cross_validation score for for fold:2 is 0.94
Cross_validation score for for fold:3 is 0.98
Cross_validation score for for fold:4 is 0.95
Cross_validation score for for fold:5 is 0.96
Cross_validation score for for fold:6 is 0.99


### 7. XGBoost Classifier <a id='section7'></a>

In [159]:
import xgboost as xgb
xgb_class = xgb.XGBClassifier()
classification_models(xgb_class, df, df_train, df_test, cv=7, features=None)

Accuracy for model: XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None) is 0.9707602339181286
Cross_validation score for for fold:0 is 0.98
Cross_validation score for for fold:1 is 0.9
Cross_validation score for for fold:2 is 0.95
Cross_validation score for for fold:3 is 0.98
Cross_validation score for for fold:4 is 0.93
Cross_validation score for for fold:5 is 0.96
Cross_validation score for for fold:6 is 0.96


## Conclusion <a id='section8'></a>
The **XGBoost** model offers the highest accuracy of all the classification models with a high cross validation score across all folds